In [1]:
import splitfolders as split

In [2]:
# Constants
SPLIT_SEED  = 2048
TRAIN_RATIO = 0.80
TEST_RATIO  = 0.05
VAL_RATIO   = 0.15

In [3]:
# Split the data
split.ratio(input = "../asl_alphabet_train/", output = "../asl_alphabet_data/", seed = SPLIT_SEED, ratio = (TRAIN_RATIO, VAL_RATIO, TEST_RATIO))

Copying files: 87000 files [02:31, 573.95 files/s]


In [6]:
import numpy as np 
from keras import models
from keras import layers
from keras import applications
from keras.applications import vgg16

In [7]:
class_names = np.zeros(27)
image_shape = (200, 200, 3)

In [8]:
base_layer = applications.VGG16(include_top = False, input_shape = image_shape, classifier_activation = None, classes = len(class_names), weights = "imagenet")
base_layer.trainable = False

base_layer.summary()

58889256/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                              

In [14]:
inputs = layers.Input(shape = image_shape)
x = vgg16.preprocess_input(inputs)
x = base_layer(x, False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(4 * len(class_names), activation = "relu")(x)
x = layers.Dense(2 * len(class_names), activation = "relu")(x)
outputs = layers.Dense(len(class_names), activation = "softmax")(x)

model = models.Model(inputs = inputs, outputs = outputs, name = "Ameri_Sign_Model")

model.summary()

Model: "Ameri_Sign_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 tf.__operators__.getitem_5   (None, 200, 200, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_5 (TFOpLambd  (None, 200, 200, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 global_average_pooling2d_5   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                  